In [1]:
#-*- coding:utf-8 -*-'
import numpy as np
import pandas as pd
from TreeNode import *

In [2]:
df = pd.DataFrame(pd.read_csv('DecisionTree.csv'))
map_Age = {'young':0,'middle-aged':1,'old':2}
map_Work = map_House = map_Label = {'no':0,'yes':1}
map_Loan = {'medium':0,'good':1,'excellent':2}
df['Age'] = df['Age'].map(map_Age)
df['Work'] = df['Work'].map(map_Work)
df['House'] = df['House'].map(map_House)
df['Loan'] = df['Loan'].map(map_Loan)
df['Label'] = df['Label'].map(map_Label)

data = df.iloc[:,1:-1].as_matrix()
label = df.iloc[:,-1].as_matrix()
label_cnt = len(map_Label)
feature_cnts = [len(map_Age), len(map_Work), len(map_House), len(map_Loan)]

In [3]:
def gini(label):
    g = 0
    for i in range(label_cnt):
        if len(label) == 0:
            return 1
        g += (len(label[label == i])/len(label))**2
    return 1-g

In [4]:
def gini_cond(data, label):
    # 遍历每个特征可能的取值，求得最小的基尼系数
    gini_cond = np.array([0.0 for _ in range(sum(feature_cnts))])
    gini_cond_idx = []
    for i in range(len(feature_cnts)):
        for j in range(feature_cnts[i]):
            gini_cond[sum(feature_cnts[0:i])+j] = \
                len(data[data[:,i] == j])/data.shape[0]*gini(label[data[:,i] == j]) + \
                len(data[data[:,i] != j])/data.shape[0]*gini(label[data[:,i] != j])
            gini_cond_idx.append((i,j))
    return gini_cond, gini_cond_idx

In [5]:
def get_max_pre_prop(label):
    prop = []
    for i in range(label_cnt):
        prop.append(len(label[label_cnt == i]))
    return np.argmax(np.array(prop))

In [7]:
CART_root = DecisionTreeNode()
CART_root.train_data = data
CART_root.label = label
CART_root.data.append(get_max_pre_prop(label))
l = [CART_root]

while l!=[]:
    current_node = l[0]
    l.remove(l[0])
    if np.sum(current_node.label-current_node.label[0]) == 0:
        # 表示可以作为叶节点
        continue
    g, g_idx = gini_cond(current_node.train_data, current_node.label)
    idx = np.argmin(g)
    current_node.data.append(g_idx[idx])
    
    node = DecisionTreeNode()
    arr_mask = current_node.train_data[:,g_idx[idx][0]] == g_idx[idx][1]
    node.train_data = current_node.train_data[arr_mask]
    node.label = current_node.label[arr_mask]
    node.data.append(get_max_pre_prop(node.label))
    current_node.addChild(node)
    l.append(node)
    
    node = DecisionTreeNode()
    arr_mask = ~arr_mask
    node.train_data = current_node.train_data[arr_mask]
    node.label = current_node.label[arr_mask]
    node.data.append(get_max_pre_prop(node.label))
    current_node.addChild(node)
    l.append(node)  
#     print(current_node.train_data.shape)
#     print(current_node.label.shape)
#     print(current_node.train_data)
#     print(current_node.label)
#     print(g)
#     print('select %s'%(str(g_idx[idx])))
#     print('===========')

In [8]:
CART_root.visualize()

选择特征：(2, 0)
选择特征：(1, 0)
